# **Instalação**

In [1]:
!pip -qqq install langchain-groq
!pip install faiss-cpu
!pip -qqq install langchain
!pip install langchain-community
!pip install sentence-transformers
!pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

# **Imports**

In [2]:
import os
import pandas as pd
import numpy as np
import re
from typing import List, Dict, Any, Optional
from datetime import datetime

# Configurações de formatação
bold_start = "\033[1m"
bold_end = "\033[0m"

# Configurar API Key (considere usar variáveis de ambiente)
os.environ["GROQ_API_KEY"] = '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'

from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from google.colab import files

# **APP RAG Nota Fiscal**

In [3]:
class NFProcessor:
    def __init__(self):
        self.df_cabecalho = None
        self.df_itens = None
        self.df_merged = None
        self.vectorstore = None
        self.llm = None
        self.qa_chain = None

        # Mapeamento das colunas reais
        self.cabecalho_columns = {
            'CHAVE DE ACESSO': 'chave_acesso',
            'NÚMERO': 'numero_nf',
            'NATUREZA DA OPERAÇÃO': 'natureza_operacao',
            'DATA EMISSÃO': 'data_emissao',
            'RAZÃO SOCIAL EMITENTE': 'emissor_razao_social',
            'UF EMITENTE': 'emissor_uf',
            'MUNICÍPIO EMITENTE': 'emissor_municipio',
            'CNPJ DESTINATÁRIO': 'destinatario_cnpj',
            'NOME DESTINATÁRIO': 'destinatario_nome',
            'UF DESTINATÁRIO': 'destinatario_uf',
            'VALOR NOTA FISCAL': 'valor_total'
        }

        self.itens_columns = {
            'CHAVE DE ACESSO': 'chave_acesso',
            'NÚMERO': 'numero_nf',
            'NÚMERO PRODUTO': 'numero_produto',
            'DESCRIÇÃO DO PRODUTO/SERVIÇO': 'descricao_produto',
            'CÓDIGO NCM/SH': 'codigo_ncm',
            'NCM/SH (TIPO DE PRODUTO)': 'tipo_produto',
            'CFOP': 'cfop',
            'QUANTIDADE': 'quantidade',
            'UNIDADE': 'unidade',
            'VALOR UNITÁRIO': 'valor_unitario',
            'VALOR TOTAL': 'valor_total_item'
        }

    def upload_and_load_files(self):
        """Carrega os arquivos CSV das notas fiscais"""
        print("📁 Fazendo upload dos arquivos...")
        _ = files.upload()

        if os.path.exists("202401_NFs_Cabecalho.csv") and os.path.exists("202401_NFs_Itens.csv"):
            print("✅ Arquivos já carregados.")
        else:
            if os.path.exists("202401_NFs.zip"):
                os.system('unzip "202401_NFs.zip" -d "."')
            else:
                print("❌ Necessário fazer upload do arquivo 202401_NFs.zip")
                return False

        try:
            self.df_cabecalho = pd.read_csv("202401_NFs_Cabecalho.csv")
            self.df_itens = pd.read_csv("202401_NFs_Itens.csv")
            print(f"✅ CSV carregados: {len(self.df_cabecalho)} cabeçalhos, {len(self.df_itens)} itens")

            # Mostrar informações dos dados
            self.analyze_data_quality()
            return True
        except FileNotFoundError as e:
            print(f"❌ Erro ao carregar CSV: {e}")
            return False

    def analyze_data_quality(self):
        """Analisa a qualidade e completude dos dados"""
        print(f"\n📊 {bold_start}Análise da qualidade dos dados:{bold_end}")

        for df_name, df in [("Cabeçalho", self.df_cabecalho), ("Itens", self.df_itens)]:
            print(f"\n{df_name}:")
            print(f"  📋 Linhas: {len(df)}")
            print(f"  📝 Colunas: {len(df.columns)}")

            # Verificar colunas principais
            key_cols = list(self.cabecalho_columns.keys()) if df_name == "Cabeçalho" else list(self.itens_columns.keys())
            available_cols = [col for col in key_cols if col in df.columns]
            print(f"  ✅ Colunas principais disponíveis: {len(available_cols)}/{len(key_cols)}")

            # Colunas com mais dados faltantes
            if len(df) > 0:
                missing_pct = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)
                print(f"  ⚠️  Top 5 colunas com dados faltantes:")
                for col, pct in missing_pct.head().items():
                    print(f"    {col}: {pct:.1f}%")

    def clean_and_preprocess_data(self):
        """Limpa e preprocessa os dados antes do merge"""
        print("🧹 Limpando e preprocessando dados...")

        # Limpar dados nulos e padronizar
        for df in [self.df_cabecalho, self.df_itens]:
            # Converter valores nulos para strings vazias
            df.fillna('', inplace=True)

            # Padronizar colunas de texto
            text_columns = df.select_dtypes(include=['object']).columns
            for col in text_columns:
                df[col] = df[col].astype(str).str.strip()
                # Remover caracteres especiais problemáticos
                df[col] = df[col].str.replace('\n', ' ').str.replace('\r', ' ')

        # Merge otimizado
        self.df_merged = pd.merge(
            self.df_itens,
            self.df_cabecalho,
            on='CHAVE DE ACESSO',
            how='left',
            suffixes=('_item', '_cabecalho')
        )

        print(f"✅ Dados mesclados: {len(self.df_merged)} registros")

        # Verificar qualidade do merge
        total_items = len(self.df_itens)
        merged_items = len(self.df_merged)
        print(f"📊 Taxa de merge: {(merged_items/total_items)*100:.1f}%")

    def create_enhanced_documents(self) -> List[Document]:
        """Cria documentos otimizados para embedding baseados nas colunas"""
        print("📝 Criando documentos otimizados...")

        documents = []

        for index, row in self.df_merged.iterrows():
            # Criar texto estruturado e otimizado
            text_parts = []

            # === INFORMAÇÕES PRINCIPAIS ===
            chave_acesso = str(row.get('CHAVE DE ACESSO', '')).strip()
            numero_nf = str(row.get('NÚMERO_item', '') or row.get('NÚMERO_cabecalho', '')).strip()

            if numero_nf:
                text_parts.append(f"Nota Fiscal {numero_nf}")
            if chave_acesso:
                text_parts.append(f"Chave: {chave_acesso}")

            # === INFORMAÇÕES DO EMISSOR ===
            emissor = str(row.get('RAZÃO SOCIAL EMITENTE', '')).strip()
            emissor_uf = str(row.get('UF EMITENTE', '')).strip()
            emissor_municipio = str(row.get('MUNICÍPIO EMITENTE', '')).strip()

            if emissor:
                text_parts.append(f"Emissor: {emissor}")
            if emissor_uf:
                text_parts.append(f"UF Emissor: {emissor_uf}")
            if emissor_municipio:
                text_parts.append(f"Município Emissor: {emissor_municipio}")

            # === INFORMAÇÕES DO DESTINATÁRIO ===
            destinatario = str(row.get('NOME DESTINATÁRIO', '')).strip()
            destinatario_uf = str(row.get('UF DESTINATÁRIO', '')).strip()
            destinatario_cnpj = str(row.get('CNPJ DESTINATÁRIO', '')).strip()

            if destinatario:
                text_parts.append(f"Destinatário: {destinatario}")
            if destinatario_uf:
                text_parts.append(f"UF Destinatário: {destinatario_uf}")
            if destinatario_cnpj:
                text_parts.append(f"CNPJ Destinatário: {destinatario_cnpj}")

            # === INFORMAÇÕES DO PRODUTO ===
            numero_produto = str(row.get('NÚMERO PRODUTO', '')).strip()
            descricao_produto = str(row.get('DESCRIÇÃO DO PRODUTO/SERVIÇO', '')).strip()
            codigo_ncm = str(row.get('CÓDIGO NCM/SH', '')).strip()
            tipo_produto = str(row.get('NCM/SH (TIPO DE PRODUTO)', '')).strip()
            quantidade = str(row.get('QUANTIDADE', '')).strip()
            unidade = str(row.get('UNIDADE', '')).strip()
            valor_unitario = str(row.get('VALOR UNITÁRIO', '')).strip()
            valor_total_item = str(row.get('VALOR TOTAL', '')).strip()

            if numero_produto:
                text_parts.append(f"Produto {numero_produto}")
            if descricao_produto:
                text_parts.append(f"Descrição: {descricao_produto}")
            if codigo_ncm:
                text_parts.append(f"NCM: {codigo_ncm}")
            if tipo_produto:
                text_parts.append(f"Tipo: {tipo_produto}")
            if quantidade and unidade:
                text_parts.append(f"Quantidade: {quantidade} {unidade}")
            if valor_unitario:
                text_parts.append(f"Valor Unitário: R$ {valor_unitario}")
            if valor_total_item:
                text_parts.append(f"Valor Total Item: R$ {valor_total_item}")

            # === INFORMAÇÕES GERAIS ===
            natureza_operacao = str(row.get('NATUREZA DA OPERAÇÃO', '')).strip()
            data_emissao = str(row.get('DATA EMISSÃO', '')).strip()
            valor_nota_fiscal = str(row.get('VALOR NOTA FISCAL', '')).strip()

            if natureza_operacao:
                text_parts.append(f"Natureza: {natureza_operacao}")
            if data_emissao:
                text_parts.append(f"Data Emissão: {data_emissao}")
            if valor_nota_fiscal:
                text_parts.append(f"Valor Total NF: R$ {valor_nota_fiscal}")

            # Criar documento
            doc_content = ". ".join([part for part in text_parts if part])

            # Metadados estruturados
            metadata = {
                "source": "merged_nfs",
                "row_index": index,
                "chave_acesso": chave_acesso,
                "numero_nf": numero_nf,
                "numero_produto": numero_produto,
                "emissor": emissor,
                "emissor_uf": emissor_uf,
                "destinatario": destinatario,
                "destinatario_uf": destinatario_uf,
                "descricao_produto": descricao_produto,
                "codigo_ncm": codigo_ncm,
                "valor_total_nf": valor_nota_fiscal,
                "valor_total_item": valor_total_item,
                "data_emissao": data_emissao,
                "natureza_operacao": natureza_operacao
            }

            documents.append(Document(page_content=doc_content, metadata=metadata))

        print(f"✅ Criados {len(documents)} documentos")
        return documents

    def create_vectorstore(self):
        """Cria o vector store com embeddings otimizados"""
        print("🔍 Criando embeddings e vector store...")

        # Usar modelo otimizado para português
        embeddings_model = SentenceTransformerEmbeddings(
            model_name="all-MiniLM-L6-v2"
        )

        documents = self.create_enhanced_documents()

        # Criar FAISS vector store
        self.vectorstore = FAISS.from_documents(documents, embeddings_model)
        print("✅ Vector store criado com sucesso")

    def setup_qa_chain(self):
        """Configura a cadeia de perguntas e respostas"""
        print("🤖 Configurando LLM e cadeia QA...")

        self.llm = ChatGroq(temperature=0, model_name="llama-3.1-8b-instant")

        # Prompt especializado para notas fiscais brasileiras
        prompt = PromptTemplate(
            template="""
            Você é um assistente especializado em análise de notas fiscais eletrônicas (NF-e) brasileiras.
            Responda à pergunta com base APENAS no contexto fornecido das notas fiscais.

            INSTRUÇÕES IMPORTANTES:
            - Responda sempre em português brasileiro
            - Seja preciso e específico
            - Sempre cite o número da nota fiscal quando disponível
            - Para valores monetários, use o formato "R$ XX,XX"
            - Se uma informação não estiver no contexto, diga "Informação não disponível"
            - Quando houver múltiplos produtos, liste todos claramente
            - Para datas, mantenha o formato original (AAAA-MM-DD)

            CONTEXTO DAS NOTAS FISCAIS:
            {context}

            PERGUNTA:
            {question}

            RESPOSTA DETALHADA:
            """,
            input_variables=["context", "question"],
        )

        llm_chain = LLMChain(
            llm=self.llm,
            prompt=prompt,
            output_parser=StrOutputParser()
        )

        self.qa_chain = StuffDocumentsChain(
            llm_chain=llm_chain,
            document_variable_name="context"
        )

        print("✅ Cadeia QA configurada")

    def search_by_filters(self, filters: Dict[str, str], limit: int = 5) -> List[Document]:
        """Busca documentos por filtros de metadata específicos"""
        if not self.vectorstore:
            return []

        # Obter todos os documentos do FAISS
        all_docs = []
        try:
            if hasattr(self.vectorstore, 'docstore') and hasattr(self.vectorstore.docstore, '_dict'):
                all_docs = list(self.vectorstore.docstore._dict.values())
        except:
            return []

        # Filtrar por metadata
        filtered_docs = []
        for doc in all_docs:
            match = True
            for key, value in filters.items():
                doc_value = str(doc.metadata.get(key, '')).strip()
                search_value = str(value).strip()

                # Busca flexível (contém)
                if search_value.lower() not in doc_value.lower():
                    match = False
                    break

            if match:
                filtered_docs.append(doc)

        return filtered_docs[:limit]

    def extract_search_params(self, query: str) -> Dict[str, Any]:
        """Extrai parâmetros de busca da pergunta"""
        params = {}

        # Buscar número de nota fiscal
        nf_patterns = [
            r"nota fiscal (?:número\s*)?(\d+)",
            r"nf (?:número\s*)?(\d+)",
            r"nota (?:número\s*)?(\d+)"
        ]

        for pattern in nf_patterns:
            match = re.search(pattern, query.lower())
            if match:
                params['numero_nf'] = match.group(1)
                break

        # Buscar número de produto
        produto_patterns = [
            r"produto (?:número\s*)?(\d+)",
            r"item (?:número\s*)?(\d+)"
        ]

        for pattern in produto_patterns:
            match = re.search(pattern, query.lower())
            if match:
                params['numero_produto'] = match.group(1)
                break

        # Buscar chave de acesso
        chave_match = re.search(r"chave.*?(\d{44})", query.lower())
        if chave_match:
            params['chave_acesso'] = chave_match.group(1)

        # Buscar empresa/emissor
        empresa_patterns = [
            r"empresa (.+?)(?:\s+e\s|\s+qual|\?|$)",
            r"emissor (.+?)(?:\s+e\s|\s+qual|\?|$)",
            r"emitente (.+?)(?:\s+e\s|\s+qual|\?|$)"
        ]

        for pattern in empresa_patterns:
            match = re.search(pattern, query.lower())
            if match:
                params['empresa'] = match.group(1).strip()
                break

        return params

    def fazer_pergunta_rag(self, query: str) -> str:
        """Método principal para fazer perguntas sobre as notas fiscais"""
        print(f"\n🔍 {bold_start}Processando pergunta:{bold_end} '{query}'")

        if not self.vectorstore or not self.qa_chain:
            return "❌ Sistema não inicializado. Execute o setup primeiro."

        # Extrair parâmetros da pergunta
        search_params = self.extract_search_params(query)
        print(f"🎯 Parâmetros identificados: {search_params}")

        retrieved_docs = []

        # Busca específica se tiver parâmetros
        if search_params:
            print("🔍 Fazendo busca específica...")

            # Buscar por número de NF
            if 'numero_nf' in search_params:
                docs = self.search_by_filters({'numero_nf': search_params['numero_nf']})
                if docs:
                    retrieved_docs.extend(docs)

            # Buscar por chave de acesso
            if 'chave_acesso' in search_params:
                docs = self.search_by_filters({'chave_acesso': search_params['chave_acesso']})
                if docs:
                    retrieved_docs.extend(docs)

            # Buscar por empresa
            if 'empresa' in search_params:
                docs = self.search_by_filters({'emissor': search_params['empresa']})
                if docs:
                    retrieved_docs.extend(docs)

        # Se não encontrou resultados específicos, fazer busca por similaridade
        if not retrieved_docs:
            print("🔍 Fazendo busca por similaridade semântica...")
            retrieved_docs = self.vectorstore.similarity_search(query, k=5)

        if not retrieved_docs:
            return "❌ Não foram encontradas informações relevantes para sua pergunta na base de dados."

        # Remover duplicatas mantendo ordem
        seen_keys = set()
        unique_docs = []
        for doc in retrieved_docs:
            key = doc.metadata.get('chave_acesso', '') + doc.metadata.get('numero_produto', '')
            if key not in seen_keys:
                seen_keys.add(key)
                unique_docs.append(doc)

        retrieved_docs = unique_docs[:5]  # Limitar a 5 documentos

        print(f"📋 {bold_start}Documentos encontrados:{bold_end} {len(retrieved_docs)}")
        for i, doc in enumerate(retrieved_docs):
            print(f"  📄 {bold_start}Doc {i+1}:{bold_end}")
            print(f"    NF: {doc.metadata.get('numero_nf', 'N/A')}")
            print(f"    Produto: {doc.metadata.get('numero_produto', 'N/A')}")
            print(f"    Emissor: {doc.metadata.get('emissor', 'N/A')[:50]}...")
            print(f"    Valor: R$ {doc.metadata.get('valor_total_item', 'N/A')}")
            print("-" * 40)

        # Gerar resposta
        try:
            response = self.qa_chain.invoke({
                "input_documents": retrieved_docs,
                "question": query
            })
            return response.get('output_text', 'Erro na geração da resposta')
        except Exception as e:
            print(f"❌ Erro ao gerar resposta: {e}")
            return "Erro ao processar a pergunta. Tente reformular."

    def gerar_relatorio_resumo(self) -> str:
        """Gera um relatório resumo dos dados carregados"""
        if self.df_merged is None:
            return "Dados não carregados"

        total_nfs = self.df_merged['CHAVE DE ACESSO'].nunique()
        total_itens = len(self.df_merged)

        # Top emissores
        top_emissores = self.df_merged['RAZÃO SOCIAL EMITENTE_item'].value_counts().head(3)

        # Valor total (se disponível)
        try:
            valor_total = self.df_merged['VALOR TOTAL'].astype(float).sum()
            valor_str = f"R$ {valor_total:,.2f}"
        except:
            valor_str = "Não disponível"

        relatorio = f"""
                    📊 RELATÓRIO RESUMO DOS DADOS

                    📈 Estatísticas Gerais:
                    • Total de Notas Fiscais: {total_nfs}
                    • Total de Itens: {total_itens}
                    • Valor Total: {valor_str}

                    🏢 Top 3 Emissores:
                    """

        for i, (emissor, count) in enumerate(top_emissores.items(), 1):
            relatorio += f"  {i}. {emissor} ({count} itens)\n"

        return relatorio



# **Execução do APP RAG Nota Fiscal**

In [4]:
# Função principal de uso
def main():
    """Função principal para executar o sistema RAG"""
    print(f"🚀 {bold_start}Iniciando Sistema RAG para Notas Fiscais{bold_end}")

    # Inicializar processador
    processor = NFProcessor()

    # Executar pipeline completo
    if not processor.upload_and_load_files():
        return None

    processor.clean_and_preprocess_data()
    processor.create_vectorstore()
    processor.setup_qa_chain()

    print(f"\n🎉 {bold_start}Sistema RAG pronto para uso!{bold_end}")

    # Mostrar relatório resumo
    print(processor.gerar_relatorio_resumo())

    # Exemplos de perguntas
    perguntas_exemplo = [
        "Qual o emissor da nota fiscal 3510129 e qual a descrição do produto número 1 nela?",
        "Quais produtos foram vendidos pela empresa COMPANHIA BRASILEIRA DE EDUC?",
        "Qual o valor total da nota fiscal 2525?",
        "Quantos produtos diferentes foram vendidos em janeiro de 2024?",
        "Qual empresa mais vendeu produtos para o governo federal?",
        "Quais são os códigos NCM mais frequentes?",
        "Qual o produto mais caro vendido?"
    ]

    print(f"\n💡 {bold_start}Exemplos de perguntas que você pode fazer:{bold_end}")
    for i, pergunta in enumerate(perguntas_exemplo, 1):
        print(f"  {i}. {pergunta}")

    # perguntas = ["Qual o emissor da nota fiscal 3510129 e qual a descrição do produto número 1 nela?", "Qual a descrição do produto número 1 da nota 2525?", "Quais produtos a COMPANHIA BRASILEIRA DE EDUC vendeu?",
    #  "Qual empresa vendeu para o FUNDO NACIONAL DE DESENVOLVIMENTO?", "Qual o produto mais caro da base?", "Quantas notas fiscais foram emitidas em São Paulo?", "Qual o total de vendas para o Mato Grosso do Sul?"]

    # # Fazer uma pergunta de exemplo
    # for pergunta in perguntas:
    #   pergunta_teste = pergunta
    #   print(f"\n🧪 {bold_start}Pergunta de exemplos:{bold_end}")
    #   resposta = processor.fazer_pergunta_rag(pergunta_teste)
    #   print(f"\n💬 {bold_start}Resposta:{bold_end}")
    #   print(resposta)

    return processor

# Para uso interativo no Colab
def fazer_pergunta_interativa(processor, pergunta):
    """Função helper para fazer perguntas de forma interativa"""
    if processor is None:
        print("❌ Execute main() primeiro para inicializar o sistema")
        return

    resposta = processor.fazer_pergunta_rag(pergunta)
    print(f"\n💬 {bold_start}Resposta:{bold_end}")
    print(resposta)
    return resposta

# Executar automaticamente
if __name__ == "__main__":
    nf_processor = main()

    # Instruções para uso
    # print(f"\n📋 {bold_start}COMO USAR:{bold_end}")
    # print("1. Para fazer perguntas, use:")
    # print("   fazer_pergunta_interativa(nf_processor, 'sua pergunta aqui')")
    # print("\n2. Ou use diretamente:")
    # print("   nf_processor.fazer_pergunta_rag('sua pergunta')")
    # print("\n3. Para ver o relatório resumo:")
    # print("   print(nf_processor.gerar_relatorio_resumo())")

🚀 Iniciando Sistema RAG para Notas Fiscais
📁 Fazendo upload dos arquivos...


Saving 202401_NFs.zip to 202401_NFs.zip
✅ CSV carregados: 100 cabeçalhos, 565 itens

📊 Análise da qualidade dos dados:

Cabeçalho:
  📋 Linhas: 100
  📝 Colunas: 21
  ✅ Colunas principais disponíveis: 11/11
  ⚠️  Top 5 colunas com dados faltantes:
    CHAVE DE ACESSO: 0.0%
    MODELO: 0.0%
    SÉRIE: 0.0%
    NÚMERO: 0.0%
    NATUREZA DA OPERAÇÃO: 0.0%

Itens:
  📋 Linhas: 565
  📝 Colunas: 27
  ✅ Colunas principais disponíveis: 11/11
  ⚠️  Top 5 colunas com dados faltantes:
    NCM/SH (TIPO DE PRODUTO): 2.3%
    MODELO: 0.0%
    CHAVE DE ACESSO: 0.0%
    NÚMERO: 0.0%
    NATUREZA DA OPERAÇÃO: 0.0%
🧹 Limpando e preprocessando dados...
✅ Dados mesclados: 565 registros
📊 Taxa de merge: 100.0%
🔍 Criando embeddings e vector store...


/tmp/ipython-input-3-2203738647.py:230: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = SentenceTransformerEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

📝 Criando documentos otimizados...
✅ Criados 565 documentos
✅ Vector store criado com sucesso
🤖 Configurando LLM e cadeia QA...
✅ Cadeia QA configurada

🎉 Sistema RAG pronto para uso!

                    📊 RELATÓRIO RESUMO DOS DADOS

                    📈 Estatísticas Gerais:
                    • Total de Notas Fiscais: 100
                    • Total de Itens: 565
                    • Valor Total: R$ 3,371,446.77

                    🏢 Top 3 Emissores:
                      1. EMPRES BRASILEIRA DE CORREIOS E TELEGRAFOS (277 itens)
  2. EDITORA FTD S.A. (44 itens)
  3. MEDTRONIC COMERCIAL LTDA. (26 itens)


💡 Exemplos de perguntas que você pode fazer:
  1. Qual o emissor da nota fiscal 3510129 e qual a descrição do produto número 1 nela?
  2. Quais produtos foram vendidos pela empresa COMPANHIA BRASILEIRA DE EDUC?
  3. Qual o valor total da nota fiscal 2525?
  4. Quantos produtos diferentes foram vendidos em janeiro de 2024?
  5. Qual empresa mais vendeu produtos para o governo fede

/tmp/ipython-input-3-2203738647.py:272: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
/tmp/ipython-input-3-2203738647.py:278: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  self.qa_chain = StuffDocumentsChain(


# **Realizar Perguntas**

In [5]:
perguntas = ["Qual o emissor da nota fiscal 3510129 e qual a descrição do produto número 1 nela?", "Qual a descrição do produto número 1 da nota 2525?", "Quais produtos a COMPANHIA BRASILEIRA DE EDUC vendeu?",
     "Qual empresa vendeu para o FUNDO NACIONAL DE DESENVOLVIMENTO?", "Qual o produto mais caro da base?", "Quantas notas fiscais foram emitidas em São Paulo?", "Qual o total de vendas para o Mato Grosso do Sul?"]

for pergunta in perguntas:
    print(f"\n🧪 {bold_start}Pergunta:{bold_end}")
    resposta = fazer_pergunta_interativa(nf_processor, pergunta)


🧪 Pergunta:

🔍 Processando pergunta: 'Qual o emissor da nota fiscal 3510129 e qual a descrição do produto número 1 nela?'
🎯 Parâmetros identificados: {'numero_nf': '3510129', 'numero_produto': '1', 'empresa': 'da nota fiscal 3510129'}
🔍 Fazendo busca específica...
📋 Documentos encontrados: 1
  📄 Doc 1:
    NF: 3510129
    Produto: 1
    Emissor: ...
    Valor: R$ 522.5
----------------------------------------

💬 Resposta:
Com base no contexto fornecido, posso responder às suas perguntas.

A nota fiscal 3510129 é emitida por um emissor que não é especificado no contexto fornecido. No entanto, é possível identificar a chave da nota fiscal, que é 41240106267630001509550010035101291224888487. Essa chave pode ser usada para identificar o emissor da nota fiscal, mas não há informações suficientes no contexto para determinar o nome do emissor.

A descrição do produto número 1 da nota fiscal 3510129 é "COLECAO SPE EF1 4ANO VOL 1 AL".

🧪 Pergunta:

🔍 Processando pergunta: 'Qual a descrição do 

In [6]:
pergunta = 'Qual fornecedor recebeu o maior valor total?'
print(f"\n🧪 {bold_start}Pergunta:{bold_end}")
resposta = fazer_pergunta_interativa(nf_processor, pergunta)


🧪 Pergunta:

🔍 Processando pergunta: 'Qual fornecedor recebeu o maior valor total?'
🎯 Parâmetros identificados: {}
🔍 Fazendo busca por similaridade semântica...
📋 Documentos encontrados: 5
  📄 Doc 1:
    NF: 776
    Produto: 1
    Emissor: ...
    Valor: R$ 8000.0
----------------------------------------
  📄 Doc 2:
    NF: 964
    Produto: 1
    Emissor: ...
    Valor: R$ 405.0
----------------------------------------
  📄 Doc 3:
    NF: 10849
    Produto: 5
    Emissor: ...
    Valor: R$ 8700.0
----------------------------------------
  📄 Doc 4:
    NF: 93
    Produto: 1
    Emissor: ...
    Valor: R$ 642.5
----------------------------------------
  📄 Doc 5:
    NF: 16967
    Produto: 28
    Emissor: ...
    Valor: R$ 5444.28
----------------------------------------

💬 Resposta:
Para responder à pergunta, precisamos calcular o valor total de cada nota fiscal e compará-los.

Nota Fiscal 776: Valor Total NF = R$ 8000,00
Nota Fiscal 964: Valor Total NF = R$ 937,50
Nota Fiscal 10849: Valo

In [7]:
pergunta = 'Qual item teve o maior volume entregue em quantidade?'
print(f"\n🧪 {bold_start}Pergunta:{bold_end}")
resposta = fazer_pergunta_interativa(nf_processor, pergunta)


🧪 Pergunta:

🔍 Processando pergunta: 'Qual item teve o maior volume entregue em quantidade?'
🎯 Parâmetros identificados: {}
🔍 Fazendo busca por similaridade semântica...
📋 Documentos encontrados: 5
  📄 Doc 1:
    NF: 93
    Produto: 1
    Emissor: ...
    Valor: R$ 642.5
----------------------------------------
  📄 Doc 2:
    NF: 10849
    Produto: 1
    Emissor: ...
    Valor: R$ 290.0
----------------------------------------
  📄 Doc 3:
    NF: 80447
    Produto: 1
    Emissor: ...
    Valor: R$ 357.57
----------------------------------------
  📄 Doc 4:
    NF: 158990
    Produto: 8
    Emissor: ...
    Valor: R$ 450.0
----------------------------------------
  📄 Doc 5:
    NF: 158990
    Produto: 7
    Emissor: ...
    Valor: R$ 450.0
----------------------------------------

💬 Resposta:
Para responder à pergunta, vamos analisar as quantidades de cada item em cada nota fiscal.

Nota Fiscal 93:
- Produto 1: 50.0 UNIDAD

Nota Fiscal 10849:
- Produto 1: 1.0 UNIDAD

Nota Fiscal 80447:
-

In [11]:
pergunta = 'Qual o produto mais caro da base?'
print(f"\n🧪 {bold_start}Pergunta:{bold_end}")
resposta = fazer_pergunta_interativa(nf_processor, pergunta)


🧪 Pergunta:

🔍 Processando pergunta: 'Qual o produto mais caro da base?'
🎯 Parâmetros identificados: {}
🔍 Fazendo busca por similaridade semântica...
📋 Documentos encontrados: 5
  📄 Doc 1:
    NF: 243008447
    Produto: 3
    Emissor: ...
    Valor: R$ 196.0
----------------------------------------
  📄 Doc 2:
    NF: 243008447
    Produto: 5
    Emissor: ...
    Valor: R$ 154.0
----------------------------------------
  📄 Doc 3:
    NF: 243010269
    Produto: 1
    Emissor: ...
    Valor: R$ 1099.8
----------------------------------------
  📄 Doc 4:
    NF: 243008447
    Produto: 4
    Emissor: ...
    Valor: R$ 315.0
----------------------------------------
  📄 Doc 5:
    NF: 243008447
    Produto: 1
    Emissor: ...
    Valor: R$ 112.08
----------------------------------------

💬 Resposta:
Para determinar o produto mais caro da base, precisamos calcular o valor total de cada produto em todas as notas fiscais fornecidas.

Nota Fiscal 243008447:
- Produto 3 (MANGA): R$ 196,00
- Produt

In [10]:
pergunta = 'Quantas notas fiscais foram emitidas em São Paulo?'
print(f"\n🧪 {bold_start}Pergunta:{bold_end}")
resposta = fazer_pergunta_interativa(nf_processor, pergunta)


🧪 Pergunta:

🔍 Processando pergunta: 'Quantas notas fiscais foram emitidas em São Paulo?'
🎯 Parâmetros identificados: {}
🔍 Fazendo busca por similaridade semântica...
📋 Documentos encontrados: 5
  📄 Doc 1:
    NF: 364908
    Produto: 24
    Emissor: ...
    Valor: R$ 6.2
----------------------------------------
  📄 Doc 2:
    NF: 728511
    Produto: 1
    Emissor: ...
    Valor: R$ 13179.6
----------------------------------------
  📄 Doc 3:
    NF: 158990
    Produto: 3
    Emissor: ...
    Valor: R$ 500.0
----------------------------------------
  📄 Doc 4:
    NF: 158990
    Produto: 4
    Emissor: ...
    Valor: R$ 500.0
----------------------------------------
  📄 Doc 5:
    NF: 369109
    Produto: 42
    Emissor: ...
    Valor: R$ 5.0
----------------------------------------

💬 Resposta:
Com base no contexto fornecido, é possível identificar que as notas fiscais foram emitidas em diferentes estados, mas não especificamente em São Paulo. No entanto, é possível identificar que a cha